In [50]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import cmath

from numpy import exp, sqrt, pi
from scipy.integrate import quad

def complex_quad(func, a, b, round=10**(-9), **kwargs):
    real_integral = quad(lambda x: np.real(func(x)), a, b, **kwargs)
    imag_integral = quad(lambda x: np.imag(func(x)), a, b, **kwargs)
    if round == 0:
        return (real_integral[0] + 1j*imag_integral[0], real_integral[1],imag_integral[1])
    else:

        if abs(real_integral[0]) < round and abs(imag_integral[0]) < round:
            return (0,real_integral[1],imag_integral[1])
        if abs(real_integral[0]) < round and not abs(imag_integral[0]) < round:
            return ( 1j*imag_integral[0], real_integral[1],imag_integral[1])
        if not abs(real_integral[0]) < round and abs(imag_integral[0]) < round:
            return (real_integral[0] , real_integral[1],imag_integral[1])
        if not abs(real_integral[0]) < round and not abs(imag_integral[0]) < round:
            return (real_integral[0] + 1j*imag_integral[0], real_integral[1],imag_integral[1])
        
    
complex_quad(lambda x: x**2, 0, 1)

(0.33333333333333337, 3.700743415417189e-15, 0.0)

In [36]:
z = complex(3,4)
np.real(z)+np.imag(z)*1j

(3+4j)

# The Standard Defining Function 
Given an integrable function $f:[a,b]\to \mathbb{C}$, it's **standard defining function** $\mathrm{std}(f)$ is the function 
\begin{align*}
\mathrm{std}(f):\mathbb{C}\setminus \mathbb{R}&\to \mathbb{C}
\\
z&\mapsto \frac{1}{2\pi i}\int_a^b \frac{f(x)}{x-z}\, dx
\end{align*}

In [43]:
def std_def(func,a,b,z):
    """
    Returns the standard defining function of a given function f(x), as a function of z
               
    """
    if func == 1:
        return (cmath.log(b-z)-cmath.log(a-z))/(2*np.pi*1j)
    else:
        return complex_quad(lambda x: func(x)/(x-z),a,b)[0]/(2*np.pi*1j)


# Hyperfunction Integration
\<insert explanation\>

In [51]:
def hyp_int(repr,a,b):
    """
    Returns the integral of a hyperfunction f(x)=[repr(z)] over the interval [a,b] 
    along the circular path whose center is (b+a)/2 and which intersects the real axis at a and b.

    Parameters
    ----------
    repr : function
        A representative of the equivalence class which defines the hyperfunction.
    a : real float
        The lower bound of the interval.
    b : real float
        The upper bound of the interval.
    """ 
    path=lambda x:((a+b)/2+((b-a)/2)*exp(1j*x))
    d_path=lambda x:((1j*(b-a)/2)*exp(1j*x))
    curve = lambda t: repr(path(t))*d_path(t)
    return -1*complex_quad(curve,0,2*np.pi)[0]

func_1 = lambda z : -1/(2*np.pi*1j*z)
func_2 = lambda z : -1/(2*np.pi*1j)*np.log(-1*z)
hyp_int(func_1,-1,1)

0.9999999999999999

Adding some functionality for matrices

In [60]:
def resolvent(A,z):
    """
    Returns the resolvent of the operator A at the point z.
    
    Parameters
    ----------
    A : numpy matrix
        The matrix representation of the operator.
    z : complex
        The point at which the resolvent is evaluated.
    """
    n = A.shape[0]
    return np.linalg.inv(A-z*np.eye(n))

def proj_measure(A,z):
    return 1/(2*np.pi*1j)*resolvent(A,z)

A = np.matrix([[1,1],[1,1]])
proj_measure(A,1)

def mat_hyp_int(A,a,b):
    """
    Returns the integral of a matrix hyperfunction f(x)=[ (2pi*i(z-A)^-1] over the interval [a,b] 
    along the circular path whose center is (b+a)/2 and which intersects the real axis at a and b.

    Parameters
    ----------
    A : numpy matrix
        The matrix representative of the equivalence class which defines the hyperfunction.
    a : real float
        The lower bound of the interval.
    b : real float
        The upper bound of the interval.
    """

    n = A.shape[0]
    M = np.zeros((n,n),dtype=complex)
    for i in range(n):
        for j in range(n):
            M[i,j] = hyp_int(lambda x: proj_measure(A,x)[i,j],a,b)
    return M
mat_hyp_int(A,-1,3)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])